In [1]:
import pandas as pd 
import numpy as np 
import requests
import json
import dotenv
import os

import sys
sys.tracebacklimit = 0

In [2]:
dotenv.load_dotenv()
spotify_client_id = os.getenv('spotify_clientid')
spotify_client_secret = os.getenv('spotify_clientsecret')


url = 'https://accounts.spotify.com/api/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded',}
data = {
    'grant_type': 'client_credentials',
    'client_id': spotify_client_id,
    'client_secret': spotify_client_secret,
}

response = requests.post(url, headers=headers, data=data)

token_info = response.json()
access_token = token_info['access_token']

In [3]:
artist_id = "4YRxDV8wJFPHPTeXepOstw" # Arijit Singh
# artist_id = "2ae6PxICSOZHvjqiCcgon8" # SPB
# artist_id = "2oSONSC9zQ4UonDKnLqksx" # Atif Aslam
# artist_id = "1Xyo4u8uXC1ZmMpatF05PJ" # The Weeknd


options = "appears_on"
limit = "50"
url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}'
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers) 
total_album_no = json.loads(response.content)['total']

album_dict = {}
print(np.arange(0,total_album_no,50))
for i in np.arange(0,total_album_no,50):
    print(i)
    offset = i
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}&offset={offset}'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers) 

    df = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
    album_dict[i] = df
album_df = pd.concat(album_dict.values())


[   0   50  100  150  200  250  300  350  400  450  500  550  600  650
  700  750  800  850  900  950 1000 1050 1100 1150 1200]
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200


In [4]:
album_df['Album_ID'] = album_df['external_urls.spotify'].apply(lambda x: x.split("/")[-1])
album_df['Producer'] = album_df['artists'].apply(lambda x: x[0]['name'])
album_df['Producer_ID'] = album_df['artists'].apply(lambda x: x[0]['id'])
album_df['name'] = album_df['name'].apply(lambda x: x.replace("(Original Motion Picture Soundtrack)",""))
album_df.to_csv("albums.csv")
album_df


,album_type,total_tracks,href,id,images,name,release_date,release_date_precision,type,uri,artists,album_group,external_urls.spotify,Album_ID,Producer,Producer_ID
0,album,12,https://api.spotify.com/v1/albums/0Rkv5iqjF2ue...,0Rkv5iqjF2uenfL0OVB8hg,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Aashiqui 2,2013-04-06,day,album,spotify:album:0Rkv5iqjF2uenfL0OVB8hg,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0Rkv5iqjF2uenfL...,0Rkv5iqjF2uenfL0OVB8hg,Mithoon,09UmIX92EUH9hAK4bxvHx6
1,album,9,https://api.spotify.com/v1/albums/2Lxoc72vRTGd...,2Lxoc72vRTGdQfMvj7Ovi1,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Yeh Jawaani Hai Deewani,2013-03-30,day,album,spotify:album:2Lxoc72vRTGdQfMvj7Ovi1,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/2Lxoc72vRTGdQfM...,2Lxoc72vRTGdQfMvj7Ovi1,Pritam,1wRPtKGflJrBx9BmLsSwlU
2,album,8,https://api.spotify.com/v1/albums/0a183xiCHiC1...,0a183xiCHiC1GQd8ou7WXO,[{'url': 'https://i.scdn.co/image/ab67616d0000...,ANIMAL,2023-11-24,day,album,spotify:album:0a183xiCHiC1GQd8ou7WXO,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0a183xiCHiC1GQd...,0a183xiCHiC1GQd8ou7WXO,Manan Bhardwaj,3pQ4aA7dkolyjUAMrVScgh
3,album,16,https://api.spotify.com/v1/albums/0LcyzKKw3RjF...,0LcyzKKw3RjFKL6ygISTeU,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Ek Tha Raja,2024-03-18,day,album,spotify:album:0LcyzKKw3RjFKL6ygISTeU,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0LcyzKKw3RjFKL6...,0LcyzKKw3RjFKL6ygISTeU,Badshah,0y59o4v8uw5crbN9M3JiL1
4,album,6,https://api.spotify.com/v1/albums/7sVzilmsqYcL...,7sVzilmsqYcLMYw2gtvoZM,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Hamari Adhuri Kahani,2015-05-21,day,album,spotify:album:7sVzilmsqYcLMYw2gtvoZM,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/7sVzilmsqYcLMYw...,7sVzilmsqYcLMYw2gtvoZM,Jeet Gannguli,2kkQthS9OLpK4UqNWYqoVl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,compilation,10,https://api.spotify.com/v1/albums/0vLCAjawZPbM...,0vLCAjawZPbMtyhR6TcGED,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Dastak - A Bouquet Of Ghazals,2015-01-01,day,album,spotify:album:0vLCAjawZPbMtyhR6TcGED,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0vLCAjawZPbMtyh...,0vLCAjawZPbMtyhR6TcGED,Various Artists,0LyfQWJT6nXafLPZqxe9Of
44,compilation,7,https://api.spotify.com/v1/albums/35MkN8qsxx5K...,35MkN8qsxx5KEELxxISsak,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Celebrating World Music Day With Indipop Chart...,2015-06-18,day,album,spotify:album:35MkN8qsxx5KEELxxISsak,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/35MkN8qsxx5KEEL...,35MkN8qsxx5KEELxxISsak,Various Artists,0LyfQWJT6nXafLPZqxe9Of
45,compilation,1,https://api.spotify.com/v1/albums/2obuaVKDXzbu...,2obuaVKDXzbuTNU9Tx0HfK,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Phir Aur Kya Chahiye (Urban Drill Mix),2023-08-11,day,album,spotify:album:2obuaVKDXzbuTNU9Tx0HfK,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/2obuaVKDXzbuTNU...,2obuaVKDXzbuTNU9Tx0HfK,Various Artists,0LyfQWJT6nXafLPZqxe9Of
46,compilation,13,https://api.spotify.com/v1/albums/1GKO9IYiDGCC...,1GKO9IYiDGCCcwWNNV0JGV,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Dance To The Tunes Turn This Party Up Vol-1,2020-09-23,day,album,spotify:album:1GKO9IYiDGCCcwWNNV0JGV,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/1GKO9IYiDGCCcwW...,1GKO9IYiDGCCcwWNNV0JGV,Various Artists,0LyfQWJT6nXafLPZqxe9Of


In [8]:
import time
all_albums = {}
for i,album_id in enumerate(album_df['Album_ID'].unique()):
    print(i,len(album_df))
    url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    try:
        response = requests.get(url, headers=headers) 
        df_t = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
    except:
        time.sleep(30)
        print("Sleeping for 30 seconds")
        response = requests.get(url, headers=headers) 
        df_t = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)


    df_t['Album_ID'] = album_id

    df_t['Artist Song'] = df_t['artists'].apply(lambda x: artist_id in str(x))
    all_albums[album_id] = df_t

df_tracks = pd.concat(all_albums.values())
df_tracks.to_csv("tracks.csv")

0 1248
1 1248
2 1248
3 1248
4 1248
5 1248
6 1248
7 1248
8 1248
9 1248
10 1248
11 1248
12 1248
13 1248
14 1248
15 1248
16 1248
17 1248
18 1248
19 1248
20 1248
21 1248
22 1248
23 1248
24 1248
25 1248
26 1248
27 1248
28 1248
29 1248
30 1248
31 1248
32 1248
33 1248
34 1248
35 1248
36 1248
37 1248
38 1248
39 1248
40 1248
41 1248
42 1248
43 1248
44 1248
45 1248
46 1248
47 1248
48 1248
49 1248
50 1248
51 1248
52 1248
53 1248
54 1248
55 1248
56 1248
57 1248
58 1248
59 1248
60 1248
61 1248
62 1248
63 1248
64 1248
65 1248
66 1248
67 1248
68 1248
69 1248
70 1248
71 1248
72 1248
73 1248
74 1248
75 1248
76 1248
77 1248
78 1248
79 1248
80 1248
81 1248
82 1248
83 1248
84 1248
85 1248
86 1248
87 1248
88 1248
89 1248
90 1248
91 1248
92 1248
93 1248
94 1248
95 1248
96 1248
97 1248
98 1248
99 1248
100 1248
101 1248
102 1248
103 1248
104 1248
105 1248
106 1248
107 1248
108 1248
109 1248
110 1248
111 1248
112 1248
113 1248
114 1248
115 1248
116 1248
117 1248
118 1248
119 1248
120 1248
121 1248
122 1248
123

In [9]:
json.loads(response.text)

{'href': 'https://api.spotify.com/v1/albums/6FWOmmRuTYal4b2UzIu2Ru/tracks?offset=0&limit=20',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2kkQthS9OLpK4UqNWYqoVl'},
     'href': 'https://api.spotify.com/v1/artists/2kkQthS9OLpK4UqNWYqoVl',
     'id': '2kkQthS9OLpK4UqNWYqoVl',
     'name': 'Jeet Gannguli',
     'type': 'artist',
     'uri': 'spotify:artist:2kkQthS9OLpK4UqNWYqoVl'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/5IhbEmJKxDv7LBDBAsrOfE'},
     'href': 'https://api.spotify.com/v1/artists/5IhbEmJKxDv7LBDBAsrOfE',
     'id': '5IhbEmJKxDv7LBDBAsrOfE',
     'name': 'Subha Dutta',
     'type': 'artist',
     'uri': 'spotify:artist:5IhbEmJKxDv7LBDBAsrOfE'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/4kq8z3vydHjPDggxb16ErB'},
     'href': 'https://api.spotify.com/v1/artists/4kq8z3vydHjPDggxb16ErB',
     'id': '4kq8z3vydHjPDggxb16ErB',
     'name': 'Nakash Aziz',
     'type': 'artist',
     'uri':

In [10]:
artist_songs = df_tracks[df_tracks['Artist Song']==True]
artist_songs = artist_songs[['artists','duration_ms','id','name','Album_ID']].rename(columns = {'name':"Track_Name",
                                                                                                'artists':"Track_Artists"})

album_details = album_df[['total_tracks','name','release_date','artists','Album_ID','Producer','Producer_ID']].rename(columns = {'name':'Album_Name',
                                                                                                                                'artists':"Album_Artists"})

final = artist_songs.merge(album_details, on = "Album_ID", how = "left", indicator=True)
print(final._merge.value_counts())
final.to_csv("All_Details_Full.csv", index = False)
final

_merge
both          2072
left_only        0
right_only       0
Name: count, dtype: int64


,Track_Artists,duration_ms,id,Track_Name,Album_ID,total_tracks,Album_Name,release_date,Album_Artists,Producer,Producer_ID,_merge
0,[{'external_urls': {'spotify': 'https://open.s...,261974,56zZ48jdyY2oDXHVnwg5Di,Tum Hi Ho,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
1,[{'external_urls': {'spotify': 'https://open.s...,304899,6UgcN95w7vQxkR8sEFmwHG,Chahun Main Ya Naa,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
2,[{'external_urls': {'spotify': 'https://open.s...,306511,55eUtE9v3uatR0y0NijM6N,Hum Mar Jayenge,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
3,[{'external_urls': {'spotify': 'https://open.s...,266069,3E9ryvrKP3hUWX79PAakH2,Meri Aashiqui,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
4,[{'external_urls': {'spotify': 'https://open.s...,214470,06zJUeKU5KFy50Cho6pDcr,Aasan Nahin Yahan,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
...,...,...,...,...,...,...,...,...,...,...,...,...
2067,[{'external_urls': {'spotify': 'https://open.s...,317413,0zciOy8kgToHnOc7m3RrAQ,Rishton Ke Saare Manzar,0vLCAjawZPbMtyhR6TcGED,10,Dastak - A Bouquet Of Ghazals,2015-01-01,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2068,[{'external_urls': {'spotify': 'https://open.s...,317348,21Nv3TlVXps943IGFWuJ8S,Rishton Ke Saare Manzar,35MkN8qsxx5KEELxxISsak,7,Celebrating World Music Day With Indipop Chart...,2015-06-18,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2069,[{'external_urls': {'spotify': 'https://open.s...,50749,3zk5QbltmuZWH8VapbdP4N,Phir Aur Kya Chahiye - Urban Drill Mix,2obuaVKDXzbuTNU9Tx0HfK,1,Phir Aur Kya Chahiye (Urban Drill Mix),2023-08-11,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2070,[{'external_urls': {'spotify': 'https://open.s...,200247,32t0LPcifv1tX4sGwrhBGE,"Hayi Hayi (From ""Dohchay"")",1GKO9IYiDGCCcwWNNV0JGV,13,Dance To The Tunes Turn This Party Up Vol-1,2020-09-23,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both


In [11]:
final_graph = final.copy()
final_graph = final_graph[final_graph['Producer']!="Various Artists"]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Best" not in x.title())]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Top" not in x.title())]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Hits" not in x.title())]

final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Remix" not in x.title())]
final_graph = final_graph[final_graph['Track_Name'].apply(lambda x: "Remix" not in x.title())]
final_graph = final_graph[final_graph['Track_Name'].apply(lambda x: "Mix" not in x.title())]
print(final_graph)
final_graph.to_csv("All_Details.csv", index = False)


                                          Track_Artists  duration_ms  \
0     [{'external_urls': {'spotify': 'https://open.s...       261974   
1     [{'external_urls': {'spotify': 'https://open.s...       304899   
2     [{'external_urls': {'spotify': 'https://open.s...       306511   
3     [{'external_urls': {'spotify': 'https://open.s...       266069   
4     [{'external_urls': {'spotify': 'https://open.s...       214470   
...                                                 ...          ...   
1947  [{'external_urls': {'spotify': 'https://open.s...       188813   
1971  [{'external_urls': {'spotify': 'https://open.s...       219193   
2000  [{'external_urls': {'spotify': 'https://open.s...       210619   
2003  [{'external_urls': {'spotify': 'https://open.s...       192400   
2064  [{'external_urls': {'spotify': 'https://open.s...       270624   

                          id                            Track_Name  \
0     56zZ48jdyY2oDXHVnwg5Di                             Tum Hi H

In [22]:
import lyricsgenius
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

genius_access_token = os.getenv('geniusclientaccesstoken')
genius = lyricsgenius.Genius(genius_access_token)

In [31]:
for i in range(final_graph.shape[0]):
    print(i)
    song_dets = final_graph.iloc[i,:].to_dict()
    # print(song_dets.keys())
    lyrics = ""
    try:
        try:
            lyrics = genius.search_song(song_dets['Track_Name'] + " Romanized").lyrics
        except:
            lyrics = genius.search_song(song_dets['Track_Name']).lyrics
            lyrics = transliterate(lyrics, sanscript.DEVANAGARI, sanscript.ITRANS)
    except: pass
        
    final_graph.loc[i,"Lyrics"] = lyrics

0
Searching for "Tum Hi Ho Romanized"...
Done.
1
Searching for "Chahun Main Ya Naa Romanized"...
Done.
2
Searching for "Hum Mar Jayenge Romanized"...
Specified song does not contain lyrics. Rejecting.
Searching for "Hum Mar Jayenge"...
Done.
3
Searching for "Meri Aashiqui Romanized"...
Done.
4
Searching for "Aasan Nahin Yahan Romanized"...
Done.
5
Searching for "Milne Hai Mujhse Aayi Romanized"...
Done.
6
Searching for "Ilahi Romanized"...
Done.
7
Searching for "Dilliwaali Girlfriend Romanized"...
No results found for: 'Dilliwaali Girlfriend Romanized'
Searching for "Dilliwaali Girlfriend"...
Done.
8
Searching for "Kabira (Encore) Romanized"...
No results found for: 'Kabira (Encore) Romanized'
Searching for "Kabira (Encore)"...
Done.
9
Searching for "Satranga Romanized"...
Done.
10
Searching for "Soulmate Romanized"...
Done.
11
Searching for "Hamari Adhuri Kahani (Title Track) Romanized"...
Done.
12
Searching for "Tujhe Kitna Chahne Lage Romanized"...
Done.
13
Searching for "Bekhayali 

In [35]:
final_graph.to_csv("All_Details.csv", index = False)